# ERCOT Electricity Market Data Cleaning
This notebook cleans and aggregates ERCOT load, price, and generation data
for downstream analysis in Power BI.

In [ ]:
import os, calendar
import pandas as pd
import numpy as np
from google.colab import files



In [ ]:
uploaded = files.upload()



KeyboardInterrupt: 

In [ ]:
xlsx_files = [f for f in os.listdir("/content") if f.endswith(".xlsx")]
xlsx_files


['Native_Load_2023.xlsx',
 'Native_Load_2025.xlsx',
 'IntGenbyFuel2025.xlsx',
 'Native_Load_2024.xlsx',
 'IntGenbyFuel2023.xlsx',
 'RPT_2024.xlsx',
 'RPT_2023.xlsx',
 'IntGenbyFuel2024.xlsx',
 'RPT_2025.xlsx']

In [ ]:
load_files = [f for f in xlsx_files if "Native_Load" in f]

dfs = [pd.read_excel(f"/content/{f}") for f in load_files]
load_raw = pd.concat(dfs, ignore_index=True)
load_raw.head()


,Hour Ending,COAST,EAST,FWEST,NORTH,NCENT,SOUTH,SCENT,WEST,ERCOT
0,01/01/2023 01:00,8514.637876,1218.506271,5073.526557,917.394049,9896.757586,2859.017922,6012.400469,1116.933155,35609.173887
1,01/01/2023 02:00,8345.632899,1179.885576,5016.774637,906.854582,9675.869720,2845.779210,5934.084260,1107.417972,35012.298856
2,01/01/2023 03:00,8125.706879,1151.144653,5016.303220,894.817013,9490.332365,2734.163642,5855.790279,1101.324245,34369.582295
3,01/01/2023 04:00,7995.083655,1153.306928,5038.003823,891.733538,9420.890470,2555.339795,5840.914307,1105.457821,34000.730337
4,01/01/2023 05:00,7906.920393,1168.779618,4994.502755,896.893843,9430.780728,2484.777679,5873.683697,1108.739956,33865.078669


In [ ]:
load_raw["Hour Ending"] = pd.to_datetime(load_raw["Hour Ending"], errors="coerce")
load_raw = load_raw.dropna(subset=["Hour Ending"])

load_raw["date"] = load_raw["Hour Ending"].dt.date
load_raw["hour"] = load_raw["Hour Ending"].dt.hour


In [ ]:
load_hourly = load_raw[["date", "hour", "ERCOT"]].rename(
    columns={"ERCOT": "load_mw"}
)

load_hourly["load_mw"] = pd.to_numeric(load_hourly["load_mw"], errors="coerce")
load_hourly = load_hourly.dropna(subset=["load_mw"])
load_hourly.head()


,date,hour,load_mw
0,2023-01-01,1,35609.173887
1,2023-01-01,2,35012.298856
2,2023-01-01,3,34369.582295
3,2023-01-01,4,34000.730337
4,2023-01-01,5,33865.078669


In [ ]:
load_hourly["date"] = pd.to_datetime(load_hourly["date"])

load_monthly = (
    load_hourly
    .groupby(pd.Grouper(key="date", freq="M"))["load_mw"]
    .mean()
    .reset_index()
    .rename(columns={"load_mw": "avg_load_mw"})
)
load_monthly.head()


/tmp/ipython-input-3225465764.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .groupby(pd.Grouper(key="date", freq="M"))["load_mw"]


,date,avg_load_mw
0,2023-01-31,43306.683752
1,2023-02-28,45083.461304
2,2023-03-31,42827.510629
3,2023-04-30,42672.360894
4,2023-05-31,49091.901775


In [ ]:
price_files = [f for f in xlsx_files if "RPT" in f.upper()]
print("Price files:", price_files)

price_dfs = []

for f in price_files:
    print(f"Processing file: {f}")
    path = f"/content/{f}"
    xls = pd.ExcelFile(path)

    for sheet in xls.sheet_names:
        if sheet.lower() in ["disclaimer", "readme"]:
            continue

        print(f"  → Sheet: {sheet}")
        try:
            df = pd.read_excel(
                path,
                sheet_name=sheet,
                usecols=[
                    "Delivery Date",
                    "Delivery Hour",
                    "Delivery Interval",
                    "Settlement Point Name",
                    "Settlement Point Price"
                ]
            )
            price_dfs.append(df)
        except Exception:
            continue

prices_raw = pd.concat(price_dfs, ignore_index=True)
prices_raw.head()


Price files: ['RPT_2024.xlsx', 'RPT_2023.xlsx', 'RPT_2025.xlsx']
Processing file: RPT_2024.xlsx
  → Sheet: Jan
  → Sheet: Feb
  → Sheet: Mar
  → Sheet: Apr
  → Sheet: May
  → Sheet: Jun
  → Sheet: Jul
  → Sheet: Aug
  → Sheet: Sep
  → Sheet: Oct
  → Sheet: Nov
  → Sheet: Dec
Processing file: RPT_2023.xlsx
  → Sheet: Jan
  → Sheet: Feb
  → Sheet: Mar
  → Sheet: Apr
  → Sheet: May
  → Sheet: Jun
  → Sheet: Jul
  → Sheet: Aug
  → Sheet: Sep
  → Sheet: Oct
  → Sheet: Nov
  → Sheet: Dec
Processing file: RPT_2025.xlsx
  → Sheet: Jan
  → Sheet: Feb
  → Sheet: Mar
  → Sheet: Apr
  → Sheet: May
  → Sheet: Jun
  → Sheet: Jul
  → Sheet: Aug
  → Sheet: Sep
  → Sheet: Oct
  → Sheet: Nov
  → Sheet: Dec


,Delivery Date,Delivery Hour,Delivery Interval,Settlement Point Name,Settlement Point Price
0,01/01/2024,1,1,HB_BUSAVG,12.77
1,01/01/2024,1,2,HB_BUSAVG,13.62
2,01/01/2024,1,3,HB_BUSAVG,14.67
3,01/01/2024,1,4,HB_BUSAVG,15.12
4,01/01/2024,1,1,HB_HOUSTON,12.58


In [ ]:
prices_raw["Delivery Date"] = pd.to_datetime(prices_raw["Delivery Date"])
prices_raw["hour"] = prices_raw["Delivery Hour"] - 1

prices_raw["datetime"] = prices_raw["Delivery Date"] + pd.to_timedelta(
    prices_raw["hour"], unit="h"
)


In [ ]:
prices_raw["price"] = pd.to_numeric(
    prices_raw["Settlement Point Price"], errors="coerce"
)
prices_raw = prices_raw.dropna(subset=["price"])

prices_monthly = (
    prices_raw
    .groupby(pd.Grouper(key="datetime", freq="M"))["price"]
    .agg(avg_price="mean", price_volatility="std")
    .reset_index()
)
prices_monthly.head()


/tmp/ipython-input-3176359787.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .groupby(pd.Grouper(key="datetime", freq="M"))["price"]


,datetime,avg_price,price_volatility
0,2023-01-31,24.187520,31.109262
1,2023-02-28,20.050350,42.304787
2,2023-03-31,27.718155,54.940186
3,2023-04-30,21.799350,39.390305
4,2023-05-31,29.980824,55.687563


In [ ]:
# ---- Find generation files ----
xlsx_files = [f for f in os.listdir("/content") if f.lower().endswith(".xlsx")]
gen_files = sorted([f for f in xlsx_files if "intgenbyfuel" in f.lower()])

if not gen_files:
    raise FileNotFoundError("❌ No IntGenbyFuel*.xlsx found in /content. Upload your IntGenbyFuel files first.")

print("✅ Found generation files:", gen_files)

month_names = list(calendar.month_abbr)[1:]  # Jan..Dec

def detect_header_row(path, sheet_name, search_rows=40):
    """Find the row index that contains header names like Date/Fuel/Total."""
    tmp = pd.read_excel(path, sheet_name=sheet_name, header=None, nrows=search_rows)
    for r in range(search_rows):
        row = tmp.iloc[r].tolist()
        vals = [str(x).strip().lower() for x in row if pd.notna(x)]
        if not vals:
            continue
        has_date = any(("date" == v) or ("date" in v) for v in vals)
        has_fuel = any(("fuel" == v) or ("fuel" in v) for v in vals)
        has_total = any(("total" == v) or ("total" in v) for v in vals)
        if has_date and has_fuel and has_total:
            return r
    return None

def standardize_cols(cols):
    return [str(c).strip().lower() for c in cols]

def pick_col_index(cols_lower, keys):
    for i, c in enumerate(cols_lower):
        for k in keys:
            if c == k or k in c:
                return i
    return None

def read_one_generation_file(path):
    xls = pd.ExcelFile(path)
    sheets = [s for s in xls.sheet_names if s in month_names]  # only months

    out_frames = []
    for sh in sheets:
        hdr = detect_header_row(path, sh)
        if hdr is None:
            # fallback guesses (some files place headers at row 0..8)
            for guess in range(0, 10):
                try:
                    df_try = pd.read_excel(path, sheet_name=sh, header=guess, nrows=5)
                    cols_lower = standardize_cols(df_try.columns)
                    if any("date" in c for c in cols_lower) and any("fuel" in c for c in cols_lower) and any("total" in c for c in cols_lower):
                        hdr = guess
                        break
                except Exception:
                    continue

        if hdr is None:
            print(f"⚠️ Skipping {os.path.basename(path)} | {sh}: could not detect header row.")
            continue

        df = pd.read_excel(path, sheet_name=sh, header=hdr)
        cols_lower = standardize_cols(df.columns)

        date_i = pick_col_index(cols_lower, ["date"])
        fuel_i = pick_col_index(cols_lower, ["fuel"])
        total_i = pick_col_index(cols_lower, ["total"])

        if date_i is None or fuel_i is None or total_i is None:
            print(f"⚠️ Skipping {os.path.basename(path)} | {sh}: missing Date/Fuel/Total. Columns: {list(df.columns)}")
            continue

        sub = df.iloc[:, [date_i, fuel_i, total_i]].copy()
        sub.columns = ["date", "fuel", "total"]
        sub["source_file"] = os.path.basename(path)
        sub["sheet"] = sh
        out_frames.append(sub)

    if not out_frames:
        return pd.DataFrame(columns=["date", "fuel", "total", "source_file", "sheet"])

    return pd.concat(out_frames, ignore_index=True)

# ---- Read all years ----
gen_raw = pd.concat(
    [read_one_generation_file(f"/content/{f}") for f in gen_files],
    ignore_index=True
)

if gen_raw.empty:
    raise ValueError("❌ No generation rows loaded. Your sheets may use different names—inspect one 'Jan' sheet manually.")

# ---- Clean types + missing values ----
gen_raw["date"] = pd.to_datetime(gen_raw["date"], errors="coerce")
gen_raw["fuel"] = gen_raw["fuel"].astype(str).str.strip()
gen_raw["total"] = pd.to_numeric(gen_raw["total"], errors="coerce").fillna(0)

gen_raw = gen_raw.dropna(subset=["date", "fuel"]).copy()

print("✅ gen_raw preview:")
display(gen_raw.head())

# ---- Daily totals by fuel ----
gen_daily = (
    gen_raw.groupby(["date", "fuel"], as_index=False)["total"]
    .sum()
    .rename(columns={"total": "gen_total"})
)

# ---- Wide daily table (columns = fuels) ----
gen_daily_wide = gen_daily.pivot_table(
    index="date",
    columns="fuel",
    values="gen_total",
    aggfunc="sum",
    fill_value=0
).reset_index()

# ---- Renewable % (robust keyword match) ----
fuel_cols = [c for c in gen_daily_wide.columns if c != "date"]
renew_cols = [c for c in fuel_cols if any(k in str(c).lower() for k in ["wind", "solar", "hydro", "biomass"])]

gen_daily_wide["total_gen"] = gen_daily_wide[fuel_cols].sum(axis=1)
gen_daily_wide["renewable_gen"] = gen_daily_wide[renew_cols].sum(axis=1) if renew_cols else 0
gen_daily_wide["renewable_pct"] = np.where(
    gen_daily_wide["total_gen"] > 0,
    gen_daily_wide["renewable_gen"] / gen_daily_wide["total_gen"],
    np.nan
)

gen_daily_wide["month"] = gen_daily_wide["date"].dt.to_period("M").astype(str)

# ---- Monthly aggregation ----
gen_monthly = (
    gen_daily_wide.groupby("month", as_index=False)
    .agg({**{c: "sum" for c in fuel_cols}, "total_gen": "sum", "renewable_gen": "sum"})
)
gen_monthly["renewable_pct"] = np.where(
    gen_monthly["total_gen"] > 0,
    gen_monthly["renewable_gen"] / gen_monthly["total_gen"],
    np.nan
)

print("✅ generation_daily preview:")
display(gen_daily_wide.head())
print("✅ generation_monthly preview:")
display(gen_monthly.head())

# ---- Export + download ----
daily_path = "/content/generation_daily_clean.csv"
monthly_path = "/content/generation_monthly_clean.csv"

gen_daily_wide.to_csv(daily_path, index=False)
gen_monthly.to_csv(monthly_path, index=False)

print("✅ Saved:", daily_path)
print("✅ Saved:", monthly_path)

files.download(daily_path)
files.download(monthly_path)


✅ Found generation files: ['IntGenbyFuel2023.xlsx', 'IntGenbyFuel2024.xlsx', 'IntGenbyFuel2025.xlsx']
✅ gen_raw preview:


,date,fuel,total,source_file,sheet
0,2023-01-01,Biomass,399.445752,IntGenbyFuel2023.xlsx,Jan
1,2023-01-01,Coal,106976.004454,IntGenbyFuel2023.xlsx,Jan
2,2023-01-01,Gas,19409.242580,IntGenbyFuel2023.xlsx,Jan
3,2023-01-01,Gas-CC,216098.972378,IntGenbyFuel2023.xlsx,Jan
4,2023-01-01,Hydro,259.665426,IntGenbyFuel2023.xlsx,Jan


✅ generation_daily preview:


fuel,date,Biomass,Coal,Gas,Gas-CC,Hydro,Nuclear,Other,Solar,WSL,Wind,total_gen,renewable_gen,renewable_pct,month
0,2023-01-01,399.445752,106976.004454,19409.242580,216098.972378,259.665426,122562.160563,1607.441246,46505.797750,-1834.513580,394469.324652,9.064535e+05,441634.233580,0.487211,2023-01
1,2023-01-02,368.646727,110342.050254,17529.048495,199358.282701,308.729914,122467.821136,1793.892119,37035.749018,-2175.315598,492462.979608,9.794919e+05,530176.105267,0.541277,2023-01
2,2023-01-03,380.300449,118731.010683,22842.942905,311800.563353,451.412134,122478.669459,2010.500894,41185.328829,-2399.267783,338310.564138,9.557920e+05,380327.605550,0.397919,2023-01
3,2023-01-04,403.496091,120598.593758,28411.720777,364044.994716,420.671682,122471.858870,1887.791741,66822.812157,-2071.088631,276793.169447,9.797840e+05,344440.149377,0.351547,2023-01
4,2023-01-05,382.785315,133314.736512,45498.751386,476691.331516,475.383960,122453.349726,1836.613645,70645.107428,-1945.798861,158783.687833,1.008136e+06,230286.964536,0.228428,2023-01


✅ generation_monthly preview:


...

✅ Saved: /content/generation_daily_clean.csv
✅ Saved: /content/generation_monthly_clean.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
gen_raw["date"] = pd.to_datetime(gen_raw["date"], errors="coerce")
gen_raw["total"] = pd.to_numeric(gen_raw["total"], errors="coerce").fillna(0)



In [ ]:
import numpy as np

# Ensure month column exists
gen_daily_wide["month"] = gen_daily_wide["date"].dt.to_period("M").astype(str)

# Select ONLY numeric columns (fuel columns + numeric metrics)
numeric_cols = gen_daily_wide.select_dtypes(include="number").columns.tolist()

# Monthly aggregation (safe: no datetime summation)
gen_monthly = (
    gen_daily_wide
    .groupby("month", as_index=False)[numeric_cols]
    .sum()
)

# OPTIONAL: recompute renewable percentage if components exist
# (robust to different fuel naming)
fuel_cols = [c for c in gen_monthly.columns if c not in ["month"]]

renew_cols = [
    c for c in fuel_cols
    if any(k in c.lower() for k in ["wind", "solar", "hydro", "biomass"])
]

if renew_cols:
    gen_monthly["total_gen"] = gen_monthly[fuel_cols].sum(axis=1)
    gen_monthly["renewable_gen"] = gen_monthly[renew_cols].sum(axis=1)
    gen_monthly["renewable_pct"] = np.where(
        gen_monthly["total_gen"] > 0,
        gen_monthly["renewable_gen"] / gen_monthly["total_gen"],
        np.nan
    )

# Preview
gen_monthly.head()


fuel,month,Biomass,Coal,Gas,Gas-CC,Hydro,Nuclear,Other,Solar,WSL,Wind,total_gen,renewable_gen,renewable_pct
0,2023-01,20531.289347,3.923907e+06,1.192343e+06,1.089448e+07,17246.504931,3.788041e+06,56986.128565,1.544600e+06,-70415.393015,1.089694e+07,3.226466e+07,1.247931e+07,0.386780
1,2023-02,32588.941026,3.576868e+06,1.277459e+06,1.029479e+07,24264.056213,3.428054e+06,56228.697465,1.473556e+06,-69092.024434,1.017473e+07,3.026944e+07,1.170514e+07,0.386698
2,2023-03,10117.304614,3.617654e+06,1.881299e+06,9.736432e+06,61030.589518,3.295830e+06,67735.042943,2.093917e+06,-83182.649782,1.106994e+07,3.175077e+07,1.323500e+07,0.416840
3,2023-04,11497.661572,3.930754e+06,1.598221e+06,9.600177e+06,53614.121903,2.715077e+06,64010.127012,2.602972e+06,-78393.976414,1.021487e+07,3.071280e+07,1.288296e+07,0.419465
4,2023-05,22572.281726,5.219683e+06,2.939097e+06,1.480470e+07,14344.160814,2.911321e+06,65006.006201,3.072015e+06,-81612.974412,7.434389e+06,3.640152e+07,1.054332e+07,0.289640


In [ ]:
import numpy as np

# Identify numeric fuel columns (everything except the month key)
fuel_cols = [c for c in gen_monthly.columns if c != "month"]

# Identify renewable columns by keyword (robust to naming differences)
renew_cols = [
    c for c in fuel_cols
    if any(k in c.lower() for k in ["wind", "solar", "hydro", "biomass"])
]

# Total generation (numeric only)
gen_monthly["total_gen"] = gen_monthly[fuel_cols].sum(axis=1)

# Renewable generation
gen_monthly["renewable_gen"] = (
    gen_monthly[renew_cols].sum(axis=1) if renew_cols else 0
)

# Renewable percentage (safe divide)
gen_monthly["renewable_pct"] = np.where(
    gen_monthly["total_gen"] > 0,
    gen_monthly["renewable_gen"] / gen_monthly["total_gen"],
    np.nan
)

# Preview
gen_monthly.head()


fuel,month,Biomass,Coal,Gas,Gas-CC,Hydro,Nuclear,Other,Solar,WSL,Wind,total_gen,renewable_gen,renewable_pct
0,2023-01,20531.289347,3.923907e+06,1.192343e+06,1.089448e+07,17246.504931,3.788041e+06,56986.128565,1.544600e+06,-70415.393015,1.089694e+07,7.700863e+07,1.247931e+07,0.162051
1,2023-02,32588.941026,3.576868e+06,1.277459e+06,1.029479e+07,24264.056213,3.428054e+06,56228.697465,1.473556e+06,-69092.024434,1.017473e+07,7.224403e+07,1.170514e+07,0.162022
2,2023-03,10117.304614,3.617654e+06,1.881299e+06,9.736432e+06,61030.589518,3.295830e+06,67735.042943,2.093917e+06,-83182.649782,1.106994e+07,7.673654e+07,1.323500e+07,0.172473
3,2023-04,11497.661572,3.930754e+06,1.598221e+06,9.600177e+06,53614.121903,2.715077e+06,64010.127012,2.602972e+06,-78393.976414,1.021487e+07,7.430856e+07,1.288296e+07,0.173371
4,2023-05,22572.281726,5.219683e+06,2.939097e+06,1.480470e+07,14344.160814,2.911321e+06,65006.006201,3.072015e+06,-81612.974412,7.434389e+06,8.334636e+07,1.054332e+07,0.126500


In [ ]:
# Quick sanity check (optional but recommended)
print("load_hourly:", load_hourly.shape)
print("load_monthly:", load_monthly.shape)
print("prices_monthly:", prices_monthly.shape)
print("gen_monthly:", gen_monthly.shape)

# Export to CSV (Power BI ready)
load_hourly.to_csv("load_hourly_clean.csv", index=False)
load_monthly.to_csv("load_monthly_clean.csv", index=False)
prices_monthly.to_csv("prices_monthly_clean.csv", index=False)
gen_monthly.to_csv("generation_monthly_clean.csv", index=False)

# Download files
files.download("load_hourly_clean.csv")
files.download("load_monthly_clean.csv")
files.download("prices_monthly_clean.csv")
files.download("generation_monthly_clean.csv")


load_hourly: (24492, 3)
load_monthly: (35, 2)
prices_monthly: (36, 3)
gen_monthly: (36, 14)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>